In [1]:
from __future__ import division, print_function, absolute_import
import pandas as pd
import tensorflow as tf
import numpy as np
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
import os
import Scalagram
from Scalagram import Scalagram
import matplotlib.pyplot as plt


In [2]:
file_location = "/Users/lucasgover/Desktop/Wavelet-Transform/Data/SwainsonCut.wav"
sg = Scalagram(file_location)
image = sg.get_data()

In [3]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
def plot_multiple_images(images, n_cols=None):
    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1
    if images.shape[-1] == 1:
        images = np.squeeze(images, axis=-1)
    plt.figure(figsize=(n_cols, n_rows))
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image, cmap="binary")
        plt.axis("off")

In [4]:
from make_stupid_mnist import get_stupid_mnist
X = get_stupid_mnist()

In [5]:
print(X[0])

[[[  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]]

 [[  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]]

 [[  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]]

 [[  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]]

 [[  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]]

 [[  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  0   0]
  [  3  18]
  [ 18  18]
  [126 136]
  [175  26]
  [166 255]
  [247 127]
  [  0

In [6]:
np.random.seed(42)
tf.random.set_seed(42)

codings_size = 30

In [7]:
generator = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[codings_size]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 14,2])
])

2023-03-03 09:38:39.505892: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [29]:
def make_generator(input_shape):  
    generator = keras.models.Sequential([
        keras.layers.Conv2D(96, 5, input_shape = input_shape, padding='same', activation='relu', name='conv1'),
        keras.layers.Conv2D(96, 2, input_shape = input_shape, padding='same', activation='relu', name='conv2'),
        keras.layers.Conv2D(96, 2, input_shape = input_shape, padding='same', activation='relu', name='conv3'),
        keras.layers.Conv2D(96, 2, input_shape = input_shape, padding='same', activation='relu', name='conv4'),
        keras.layers.MaxPool2D(pool_size=1, padding='same', name='pool4'),
        keras.layers.Reshape([input_shape[0],None]),
        keras.layers.Bidirectional(keras.layers.LSTM(units=256, return_sequences=True), name='bi_lstm1'),
        keras.layers.Bidirectional(keras.layers.LSTM(units=256, return_sequences=True), name='bi_lstm2'),
        keras.layers.Reshape([input_shape]),
        
    ])
    generator.summary()
    return generator

In [31]:
from RNN_GAN import make_discriminator
generator = make_generator((28,None,2))
discriminator = make_discriminator((28,None,2))

ValueError: Exception encountered when calling layer "reshape_13" (type Reshape).

Tried to convert 'shape' to a tensor and failed. Error: None values not supported.

Call arguments received:
  • inputs=tf.Tensor(shape=(None, 28, None, 96), dtype=float32)

In [ ]:
gan = keras.models.Sequential([generator, discriminator])

ValueError: Exception encountered when calling layer "sequential_12" (type Sequential).

Input 0 of layer "conv1" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 28, 512)

Call arguments received:
  • inputs=tf.Tensor(shape=(None, 28, 512), dtype=float32)
  • training=None
  • mask=None

In [ ]:
discriminator.compile(loss="binary_crossentropy", optimizer="adam")
discriminator.trainable = True
gan.compile(loss="binary_crossentropy", optimizer="adam")

In [ ]:
batch_size = 32
print("X_train type " + str(type(X)))
dataset = tf.data.Dataset.from_tensor_slices(X).shuffle(1000)

X_train type <class 'tuple'>


In [ ]:
print(type(dataset))
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)
print(type(dataset))

<class 'tensorflow.python.data.ops.dataset_ops.ShuffleDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>


In [ ]:
def makeImages():
    noise = tf.random.normal(shape=[batch_size, codings_size])
    print(noise.shape)
    generated_images = generator(noise)
    generated_images = generated_images.numpy().reshape(batch_size,28,28)
    plot_multiple_images(generated_images, 8)
makeImages()

KeyboardInterrupt: 

In [ ]:
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        print("Epoch {}/{}".format(epoch + 1, n_epochs))              # not shown in the book
        for X_batch in dataset:
            # phase 1 - training the discriminator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            generated_images = generator(noise)
            X_batch = tf.cast(X_batch, tf.float32)
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)
            generated_images = generated_images.numpy().reshape(batch_size,28,28)
            makeImages()                     # not shown
            plt.show()
train_gan(gan, dataset, batch_size, codings_size, n_epochs=20)

Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
